In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import os
import sys
import shutil
import math
import random
import heapq 
import time
import copy
import itertools  
from PIL import Image
from io import StringIO,BytesIO 
from scipy.spatial.distance import pdist
import cv2
from scipy.signal import butter, lfilter
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,roc_curve,accuracy_score,auc 
from functools import reduce
import wfdb#https://github.com/MIT-LCP/wfdb-python
from wfdb import processing
import faiss 
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
torch.cuda.set_device(1)
print (torch.cuda.current_device())

Loading faiss with AVX2 support.


1


In [2]:
#read train image with CV
train_dir = '/data/fjsdata/ECG/MIT-BIH/train' #the path of images
trN, trI, trY = [],[],[]
for iname in os.listdir(train_dir):
    if iname.endswith(".png"):
        try:
            image_path = os.path.join(train_dir, iname)
            itype = int(os.path.splitext(iname)[0].split("-")[1])
            img = cv2.resize(cv2.imread(image_path).astype(np.float32), (256, 256))#(500,300,3)->(256,256,3)
            trN.append(iname)
            trI.append(img)
            trY.append(itype)
        except:
            print(iname+":"+str(image_path))
        sys.stdout.write('\r{} / {} '.format(len(trY),20000))
        sys.stdout.flush()
print('The length of train set is %d'%len(trY))
#read test image with CV
test_dir = '/data/fjsdata/ECG/MIT-BIH/test' #the path of images
teN, teI, teY = [],[],[]
for iname in os.listdir(test_dir):
    if iname.endswith(".png"):
        try:
            image_path = os.path.join(test_dir, iname)
            itype = int(os.path.splitext(iname)[0].split("-")[1])
            img = cv2.resize(cv2.imread(image_path).astype(np.float32), (256, 256))#(500,300,3)->(256,256,3)
            teN.append(iname)
            teI.append(img)
            teY.append(itype)
        except:
            print(iname+":"+str(image_path))
        sys.stdout.write('\r{} / {} '.format(len(teY),20000))
        sys.stdout.flush()
print('The length of test set is %d'%len(teY))

20000 / 20000 The length of train set is 20000
20000 / 20000 The length of test set is 20000


In [3]:
class SpatialAttention(nn.Module):#spatial attention layer
    def __init__(self):
        super(SpatialAttention, self).__init__()

        self.conv1 = nn.Conv2d(2, 1, kernel_size=3, padding=1, bias=False)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        x = torch.cat([avg_out, max_out], dim=1)
        x = self.conv1(x)
        return self.sigmoid(x)
    
class ResBlock(nn.Module):
    def __init__(self, in_channels: int, out_channels: int, stride=1):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(
                in_channels=in_channels, out_channels=out_channels,
                kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, 3, 1, 1, bias=False),
            nn.BatchNorm2d(out_channels),
        )

        self.downsample_layer = None
        self.do_downsample = False
        if in_channels != out_channels or stride != 1:
            self.do_downsample = True
            self.downsample_layer = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, 3, stride, 1, bias=False),
                nn.BatchNorm2d(out_channels),
            )

        # initialize weights
        self.apply(self.init_weights)

    def forward(self, x):
        identity = x
        out = self.net(x)

        if self.do_downsample:
            identity = self.downsample_layer(x)

        return F.relu(out + identity, inplace=True) #resnet

    @staticmethod
    def init_weights(m):
        if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
            nn.init.xavier_normal_(m.weight)
            
class ATHNet(nn.Module):
    def __init__(self, hash_size: int, type_size: int):
        super().__init__()
        #resnet and maxpool
        self.net1 = nn.Sequential(#(3,256,256)->(16,128,128)
            ResBlock(in_channels=3, out_channels=16, stride=2), 
            nn.MaxPool2d(kernel_size=3, padding=1, stride=1)
        )
        
        #Attention (16,128,128)->(16,128,128)
        self.sa = SpatialAttention()
        
        #resnet and meanpool
        self.net2 =nn.Sequential( #(16,128,128)->(8,64,64)
            ResBlock(in_channels=16, out_channels=8, stride=2),
            nn.AvgPool2d(kernel_size=3, padding=1, stride=1)
        ) 
         
        #fully connected with conv (8,64,64)->(1,32,32)
        self.dense=ResBlock(in_channels=8, out_channels=1, stride=2)
        #fully connected (1,32,32)->class_size
        self.hashlayer = nn.Linear(1*32*32, hash_size)
        self.typelayer = nn.Linear(1*32*32, type_size)
    
        # initialize weights
        self.apply(self.init_weights)

    def forward(self, x):
        x = self.net1(x)
        x = self.sa(x)*x
        x = self.net2(x)
        x = self.dense(x)
        x = x.view(x.size(0),-1)
        x_hash = self.hashlayer(x)
        x_type = self.typelayer(x)
        return x_hash, x_type

    @staticmethod
    def init_weights(m):
        if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
            nn.init.xavier_normal_(m.weight)

class HashLossFunc(nn.Module):
    def __init__(self, margin=0.5, alpha=0.01):
        super(HashLossFunc, self).__init__()
        self.alpha = alpha #regularization
        self.margin = margin #margin threshold
        self.mse_loss = nn.MSELoss(reduction='none')
        self.l1_loss = nn.L1Loss(reduction='mean')
    
    def forward(self,h1,h2,y):    
        margin_val = self.margin * h1.shape[1]
        squared_loss = torch.mean(self.mse_loss(h1, h2), dim=1)
        # T1: 0.5 * (1 - y) * dist(x1, x2)
        positive_pair_loss = (0.5 * (1 - y) * squared_loss)
        mean_positive_pair_loss = torch.mean(positive_pair_loss)
        # T2: 0.5 * y * max(margin - dist(x1, x2), 0)
        zeros = torch.zeros_like(squared_loss)
        marginMat = margin_val * torch.ones_like(squared_loss)
        negative_pair_loss = 0.5 * y * torch.max(zeros, marginMat - squared_loss)
        mean_negative_pair_loss = torch.mean(negative_pair_loss)

        # T3: alpha(dst_l1(abs(x1), 1)) + dist_l1(abs(x2), 1)))
        mean_value_regularization = self.alpha * (
                self.l1_loss(torch.abs(h1), torch.ones_like(h1)) +
                self.l1_loss(torch.abs(h2), torch.ones_like(h2)))

        loss = mean_positive_pair_loss + mean_negative_pair_loss + mean_value_regularization
        return loss
    
#https://github.com/luyajie/triplet-deep-hash-pytorch#triplet-deep-hash-pytorch            
class TripletLoss(nn.Module):
    def __init__(self, margin=0.5):
        super(TripletLoss, self).__init__()
        self.margin = margin #margin threshold
        self.mse_loss = nn.MSELoss(reduction='none')
    
    def forward(self,H_q,H_p,H_n):    
        margin_val = self.margin * H_q.shape[1]
        squared_loss_pos = torch.mean(self.mse_loss(H_q, H_p), dim=1)
        squared_loss_neg = torch.mean(self.mse_loss(H_q, H_n), dim=1)
        zeros = torch.zeros_like(squared_loss_neg)
        loss  = torch.max(zeros, margin_val - squared_loss_neg + squared_loss_pos)
        return torch.mean(loss)

class FocalLoss(nn.Module):
    #Loss(x, class) = - \alpha (1-softmax(x)[class])^gamma \log(softmax(x)[class])
    def __init__(self, gamma=0, alpha=None, size_average=True):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha
        if isinstance(alpha,(float,int)): self.alpha = torch.Tensor([alpha,1-alpha])
        if isinstance(alpha,list): self.alpha = torch.Tensor(alpha)
        self.size_average = size_average

    def forward(self, out, y):
        y = y.view(-1,1)
        logpt = F.log_softmax(out,dim=1)#default ,dim=1
        logpt = logpt.gather(1,y)# dim=1, index=y, max
        logpt = logpt.view(-1)
        pt = Variable(logpt.data.exp())

        if self.alpha is not None:
            if self.alpha.type()!=out.data.type():
                self.alpha = self.alpha.type_as(out.data)
            at = self.alpha.gather(0,y.data.view(-1))
            logpt = logpt * Variable(at)

        loss = -1 * (1-pt)**self.gamma * logpt
        if self.size_average: return loss.mean()
        else: return loss.sum()

class CircleLoss(nn.Module):
    def __init__(self, scale=32, margin=0.25, similarity='cos', **kwargs):
        super(CircleLoss, self).__init__()
        self.scale = scale
        self.margin = margin
        self.similarity = similarity

    def forward(self, feats, labels):
        assert feats.size(0) == labels.size(0), \
            f"feats.size(0): {feats.size(0)} is not equal to labels.size(0): {labels.size(0)}"
        batch_size = feats.size(0)
        if self.similarity == 'dot':
            sim_mat = torch.matmul(feats, torch.t(feats))
        elif self.similarity == 'cos':
            feats = F.normalize(feats)
            sim_mat = feats.mm(feats.t())
        else:
            raise ValueError('This similarity is not implemented.')
        loss = list()
        for i in range(batch_size):
            pos_index = labels == labels[i]
            pos_index[i] = 0
            neg_index = labels != labels[i]
            pos_pair_ = sim_mat[i][pos_index]
            neg_pair_ = sim_mat[i][neg_index]

            alpha_p = torch.relu(-pos_pair_ + 1 + self.margin)
            alpha_n = torch.relu(neg_pair_ + self.margin)
            margin_p = 1 - self.margin
            margin_n = self.margin
            loss_p = torch.sum(torch.exp(-self.scale * alpha_p * (pos_pair_ - margin_p)))
            loss_n = torch.sum(torch.exp(self.scale * alpha_n * (neg_pair_ - margin_n)))
            loss.append(torch.log(1 + loss_p * loss_n))

        loss = sum(loss) / batch_size
        return loss

#Generate image pairs for model
def onlineGenImgPairs():
    if (len(trY) % 2) == 0: spls = len(trY)
    else:  spls = len(trY)-1
    idx_sf = random.sample(range(0, spls),spls)
    trI1_sf, trI2_sf, trY1_sf, trY2_sf = [],[],[],[]
    flag = 0
    for i in idx_sf:
        if flag==0:
            trI1_sf.append(trI[i])
            trY1_sf.append(trY[i])
            flag =1
        else:
            trI2_sf.append(trI[i])
            trY2_sf.append(trY[i])
            flag =0
    trY_sf = np.where((np.array(trY1_sf)-np.array(trY2_sf))!=0,1,0)
    return np.array(trI1_sf),np.array(trI2_sf),trY_sf

#Generate image pairs for model
def onlineGenImgTriplets( ):
    idx_sf = []
    idx_0 = np.where( np.array(trY) == 0 ) #class 0
    idx_0 = list(idx_0[0])[0:4555]
    idx_sf.extend(idx_0)
    idx_1 = np.where( np.array(trY) == 1 ) #class 1
    idx_1 = list(idx_1[0])
    idx_sf.extend(idx_1)
    idx_2 = np.where( np.array(trY) == 2 ) #class 2
    idx_2 = list(idx_2[0])
    idx_sf.extend(idx_2)
    idx_3 = np.where( np.array(trY) == 3 ) #class 3
    idx_3 = list(idx_3[0])
    idx_sf.extend(idx_3)
    random.shuffle(idx_sf)   
    trQ_sf, trP_sf, trN_sf = [], [], []
    trQ_y, trP_y, trN_y = [], [], []
    for iQ in idx_sf:
        trQ_sf.append(trI[iQ])
        trQ_y.append(trY[iQ])
        if trY[iQ] == 0:
            idx_tmp = idx_0.copy()
            idx_tmp.remove(iQ)
            iP =  random.sample(idx_tmp,1) #remove self,then get one positive sample
            trP_sf.append(trI[iP[0]])
            trP_y.append(trY[iP[0]])
            idx_sf_tmp = list(set(idx_sf) - set(idx_0))
            iN =  random.sample(idx_sf_tmp,1) #remove positive and get one negative sample
            trN_sf.append(trI[iN[0]])
            trN_y.append(trY[iN[0]])
        elif trY[iQ] == 1:
            idx_tmp = idx_1.copy()
            idx_tmp.remove(iQ)
            iP =  random.sample(idx_tmp,1) #remove self,then get one positive sample
            trP_sf.append(trI[iP[0]])
            trP_y.append(trY[iP[0]])
            idx_sf_tmp = list(set(idx_sf) - set(idx_1))
            iN =  random.sample(idx_sf_tmp,1) #remove positive and get one negative sample
            trN_sf.append(trI[iN[0]])
            trN_y.append(trY[iN[0]])
        elif trY[iQ] == 2:
            idx_tmp = idx_2.copy()
            idx_tmp.remove(iQ)
            iP =  random.sample(idx_tmp,1) #remove self,then get one positive sample
            trP_sf.append(trI[iP[0]])
            trP_y.append(trY[iP[0]])
            idx_sf_tmp = list(set(idx_sf) - set(idx_2))
            iN =  random.sample(idx_sf_tmp,1) #remove positive and get one negative sample
            trN_sf.append(trI[iN[0]])
            trN_y.append(trY[iN[0]])
        elif trY[iQ] == 3:
            idx_tmp = idx_3.copy()
            idx_tmp.remove(iQ)
            iP =  random.sample(idx_tmp,1) #remove self,then get one positive sample
            trP_sf.append(trI[iP[0]])
            trP_y.append(trY[iP[0]])
            idx_sf_tmp = list(set(idx_sf) - set(idx_3))
            iN =  random.sample(idx_sf_tmp,1) #remove positive and get one negative sample
            trN_sf.append(trI[iN[0]])
            trN_y.append(trY[iN[0]])
        else: pass
        sys.stdout.write('\r{} / {} '.format(len(trQ_sf),len(idx_sf)))
        sys.stdout.flush()
    return np.array(trQ_sf),np.array(trP_sf),np.array(trN_sf), np.array(trQ_y), np.array(trP_y), np.array(trN_y)

In [8]:
#trQ_sf, trP_sf, trN_sf, trQ_y, trP_y, trN_y = onlineGenImgTriplets() #sample  triplet labels
assert (trQ_sf.shape==trP_sf.shape and trQ_sf.shape==trN_sf.shape)
assert (trQ_y.shape==trP_y.shape and trQ_y.shape==trN_y.shape)
assert (np.mean(np.where((np.array(trQ_y)-np.array(trP_y))!=0,1,0))==0.0)
assert (np.mean(np.where((np.array(trQ_y)-np.array(trN_y))!=0,1,0))==1.0)

#define model
model = ATHNet(hash_size=36, type_size=4).cuda()#initialize model
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) #define optimizer
tl_loss  = TripletLoss(margin=0.5).cuda() #define TripletLoss 
ce_loss  = nn.CrossEntropyLoss().cuda() #define ce mutli-classes

#train model
best_net, best_loss = None, float('inf')
batchSize = 10
for epoch in range(10):#iteration
    losses = []
    shuffled_idx = np.random.permutation(np.arange(len(trQ_sf)))
    train_q = trQ_sf[shuffled_idx]
    train_q_y = trQ_y[shuffled_idx]
    train_p = trP_sf[shuffled_idx]
    train_p_y = trP_y[shuffled_idx]
    train_n = trN_sf[shuffled_idx]
    train_n_y = trN_y[shuffled_idx]
    num_batches = len(trQ_sf) // batchSize
    for i in range(num_batches):
        optimizer.zero_grad()#grad vanish
        min_idx = i * batchSize
        max_idx = np.min([len(trQ_sf), (i+1)*batchSize])
        Q_batch = torch.from_numpy(train_q[min_idx:max_idx]).type(torch.FloatTensor).cuda()
        Q_y_batch = torch.from_numpy(train_q_y[min_idx:max_idx]).type(torch.LongTensor).cuda()
        P_batch = torch.from_numpy(train_p[min_idx:max_idx]).type(torch.FloatTensor).cuda()
        P_y_batch = torch.from_numpy(train_p_y[min_idx:max_idx]).type(torch.LongTensor).cuda()
        N_batch = torch.from_numpy(train_n[min_idx:max_idx]).type(torch.FloatTensor).cuda()
        N_y_batch = torch.from_numpy(train_n_y[min_idx:max_idx]).type(torch.LongTensor).cuda()
        #forword
        Q_hash, Q_type = model(Q_batch.permute(0, 3, 1, 2))#permute the dims of matrix
        P_hash, P_type = model(P_batch.permute(0, 3, 1, 2))
        N_hash, N_type = model(N_batch.permute(0, 3, 1, 2))
        #loss
        hash_loss = tl_loss(Q_hash,P_hash,N_hash)
        type_loss = ce_loss(Q_type,Q_y_batch) + ce_loss(P_type,P_y_batch) + ce_loss(N_type,N_y_batch) #F.log_softmax+F.nll_loss
        loss = hash_loss+type_loss
        #backward
        loss.backward()
        #update parameters
        optimizer.step()
        #show loss
        sys.stdout.write('\r {} / {} : loss = {}'.format(i+1, num_batches, float('%0.6f'%loss.item())))
        sys.stdout.flush()     
        losses.append(loss.item())
    print("Eopch: %5d mean_loss = %.6f" % (epoch + 1, np.mean(losses)))
    if np.mean(losses) < best_loss:
        best_loss = np.mean(losses)
        best_net = copy.deepcopy(model)
print("best_loss = %.6f" % (best_loss))
#release gpu memory
model = model.cpu()
tl_loss=tl_loss.cpu()
ce_loss=ce_loss.cpu()
torch.cuda.empty_cache()

#hash code of train data from model
batchSize = 10
num_batches = len(trI) // batchSize
trF = []
for i in range(num_batches):
    min_idx = i * batchSize
    max_idx = np.min([len(trI), (i+1)*batchSize])
    I_batch = torch.from_numpy(np.array(trI[min_idx: max_idx])).type(torch.FloatTensor).cuda()
    X_batch,_ = best_net(I_batch.permute(0, 3, 1, 2))#forword
    I_batch = I_batch.cpu()
    X_batch = X_batch.cpu()
    torch.cuda.empty_cache()#release gpu memory
    trF.extend(X_batch.data.numpy().tolist())
    sys.stdout.write('\r {} / {} '.format(i, num_batches))
    sys.stdout.flush()
    
#hash code of test data from model
teY_pred = []
teF = [] 
num_batches = len(teY) // batchSize 
for i in range(num_batches):
    min_idx = i * batchSize
    max_idx = np.min([len(teY), (i+1)*batchSize])
    x_batch = torch.from_numpy(np.array(teI[min_idx:max_idx])).type(torch.FloatTensor).cuda()
    x_hash, x_type = best_net(x_batch.permute(0, 3, 1, 2))#forword
    teF.extend(x_hash.cpu().data.numpy().tolist()) #record feature
    x_type = F.log_softmax(x_type,dim=1) 
    pred = x_type.max(1,keepdim=True)[1]
    teY_pred.extend(pred.cpu().data.numpy().tolist())
    sys.stdout.write('\r {} / {} '.format(i, num_batches))
    sys.stdout.flush()

#performance of retrieval
# buliding index of trainset
tstart = time.time()
cpu_index = faiss.IndexFlatL2(36) #
gpu_index = faiss.index_cpu_to_all_gpus(cpu_index) #make all gpu usable
gpu_index.add(np.ascontiguousarray(trF, dtype=np.float32)) #add data(must be float32) to index
elapsed = time.time() - tstart    
print('Completed buliding index in %d seconds' % int(elapsed))
for topk in [5,10,15,20]:
    MHR = [] #mean Hit ratio 
    MAP = [] #mean average precision
    MRR = [] #mean reciprocal rank
    scores, neighbors = gpu_index.search(np.array(teF).astype('float32'), k=topk)
    for i, teVal in enumerate(teF):
        stype = teY[i]
        #perfromance
        pos_len = 0
        rank_len = 0
        mrr_flag = 0
        #for j in ranklist:
        for j in neighbors[i].tolist():
            dtype = trY[j]
            rank_len=rank_len+1
            if stype==dtype:  #hit
                MHR.append(1)
                pos_len = pos_len +1
                MAP.append(pos_len/rank_len) 
                if mrr_flag==0: 
                    MRR.append(pos_len/rank_len)
                    mrr_flag =1
            else: 
                MHR.append(0)
                MAP.append(0)   
    print("topk={}, mHR={:.6f}, mAP={:.6f}, mRR={:.6f}".format(topk, np.mean(MHR),np.mean(MAP),np.mean(MRR)))
#performance of classification
print ( 'Accuracy: %.6f'%accuracy_score(teY, teY_pred))
labels = list(set(teY))
cm = confusion_matrix(teY, teY_pred, labels=labels )
print (cm)
print ('Sensitivity of Normal: %.6f'%float(cm[0][0]/np.sum(cm[0])))
print ('Sensitivity of Edema: %.6f'%float(cm[1][1]/np.sum(cm[1])))
print ('Sensitivity of Pneumonia: %.6f'%float(cm[2][2]/np.sum(cm[2])))
print ('Sensitivity of Fracture: %.6f'%float(cm[3][3]/np.sum(cm[3])))

 1000 / 1000 : loss = 1.476945Eopch:     1 mean_loss = 4.266039
 1000 / 1000 : loss = 2.122481Eopch:     2 mean_loss = 2.482491
 1000 / 1000 : loss = 5.973835Eopch:     3 mean_loss = 2.081608
 1000 / 1000 : loss = 0.398822Eopch:     4 mean_loss = 1.787284
 1000 / 1000 : loss = 0.262652Eopch:     5 mean_loss = 1.572742
 1000 / 1000 : loss = 2.074561Eopch:     6 mean_loss = 1.387705
 1000 / 1000 : loss = 0.105523Eopch:     7 mean_loss = 1.196310
 1000 / 1000 : loss = 0.244838Eopch:     8 mean_loss = 0.991722
 1000 / 1000 : loss = 12.335144Eopch:     9 mean_loss = 0.873514
 1000 / 1000 : loss = 0.271365Eopch:    10 mean_loss = 0.779182
best_loss = 0.779182
 1999 / 2000 Completed buliding index in 22 seconds
topk=5, mHR=0.750120, mAP=0.722837, mRR=0.926407
topk=10, mHR=0.753030, mAP=0.716481, mRR=0.896289
topk=15, mHR=0.755083, mAP=0.714177, mRR=0.881909
topk=20, mHR=0.755822, mAP=0.712810, mRR=0.873714
Accuracy: 0.721600
[[10410  1003  3100   341]
 [  227   433   267    17]
 [  109   117 

In [9]:
for cls in labels:
    teY_cls = np.array(teY)[np.where( np.array(teY) == cls)[0]]
    teF_cls = np.array(teF)[np.where( np.array(teY) == cls)[0]]
    MHR = [] #mean Hit ratio 
    MAP = [] #mean average precision
    MRR = 0.0 #mean reciprocal rank
    scores, neighbors = gpu_index.search(np.array(teF_cls).astype('float32'), k=10)
    for i, teVal in enumerate(teF_cls):#perfromance
        stype = teY_cls[i]
        pos_len = 0
        rank_len = 0
        mrr_flag = 0
        for j in neighbors[i].tolist():
            dtype = trY[j]
            rank_len=rank_len+1
            if stype==dtype:  #hit
                MHR.append(1)
                pos_len = pos_len +1
                MAP.append(pos_len/rank_len) 
                if mrr_flag==0: 
                    MRR= pos_len/rank_len
                    mrr_flag = 1
            else: 
                MHR.append(0)
                MAP.append(0)   
    print("label={}, mHR={:.6f}, mAP={:.6f}, mRR={:.6f}".format(cls, np.mean(MHR),np.mean(MAP),np.mean(MRR)))

label=0, mHR=0.768332, mAP=0.734026, mRR=1.000000
label=1, mHR=0.331674, mAP=0.256860, mRR=0.333333
label=2, mHR=0.874340, mAP=0.836893, mRR=1.000000
label=3, mHR=0.054831, mAP=0.033287, mRR=0.142857


In [12]:
#define model
model = ATHNet(hash_size=36, type_size=4).cuda()#initialize model
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) #define optimizer
tl_loss  = TripletLoss(margin=0.5).cuda() #define TripletLoss 

#train model
best_net, best_loss = None, float('inf')
batchSize = 10
pl_t_i = []
for epoch in range(10):#iteration
    losses = []
    shuffled_idx = np.random.permutation(np.arange(len(trQ_sf)))
    train_q = trQ_sf[shuffled_idx]
    train_p = trP_sf[shuffled_idx]
    train_n = trN_sf[shuffled_idx]
    num_batches = len(trQ_sf) // batchSize
    for i in range(num_batches):
        optimizer.zero_grad()#grad vanish
        min_idx = i * batchSize
        max_idx = np.min([len(trQ_sf), (i+1)*batchSize])
        Q_batch = torch.from_numpy(train_q[min_idx:max_idx]).type(torch.FloatTensor).cuda()
        P_batch = torch.from_numpy(train_p[min_idx:max_idx]).type(torch.FloatTensor).cuda()
        N_batch = torch.from_numpy(train_n[min_idx:max_idx]).type(torch.FloatTensor).cuda()
        #forword
        Q_hash, _ = model(Q_batch.permute(0, 3, 1, 2))#permute the dims of matrix
        P_hash, _ = model(P_batch.permute(0, 3, 1, 2))
        N_hash, _ = model(N_batch.permute(0, 3, 1, 2))
        #loss
        loss = tl_loss(Q_hash,P_hash,N_hash)
        #backward
        loss.backward()
        #update parameters
        optimizer.step()
        #show loss
        sys.stdout.write('\r {} / {} : loss = {}'.format(i+1, num_batches, float('%0.6f'%loss.item())))
        sys.stdout.flush()     
        losses.append(loss.item())
    print("Eopch: %5d mean_loss = %.6f" % (epoch + 1, np.mean(losses)))
    pl_t_i.append(np.mean(losses))
    if np.mean(losses) < best_loss:
        best_loss = np.mean(losses)
        best_net = copy.deepcopy(model)
print("best_loss = %.6f" % (best_loss))
#release gpu memory
model = model.cpu()
tl_loss=tl_loss.cpu()
torch.cuda.empty_cache()

#hash code of train data from model
batchSize = 10
num_batches = len(trI) // batchSize 
trF = []
for i in range(num_batches):
    min_idx = i * batchSize
    max_idx = np.min([len(trI), (i+1)*batchSize])
    I_batch = torch.from_numpy(np.array(trI[min_idx: max_idx])).type(torch.FloatTensor).cuda()
    X_batch,_ = best_net(I_batch.permute(0, 3, 1, 2))#forword
    I_batch = I_batch.cpu()
    X_batch = X_batch.cpu()
    torch.cuda.empty_cache()#release gpu memory
    trF.extend(X_batch.data.numpy().tolist())
    sys.stdout.write('\r {} / {} '.format(i, num_batches))
    sys.stdout.flush()
    
#hash code of test data from model
teY_pred = []
teY_prob = []
teF = [] 
num_batches = len(teY) // batchSize 
for i in range(num_batches):
    min_idx = i * batchSize
    max_idx = np.min([len(teY), (i+1)*batchSize])
    x_batch = torch.from_numpy(np.array(teI[min_idx:max_idx])).type(torch.FloatTensor).cuda()
    x_hash, x_type = best_net(x_batch.permute(0, 3, 1, 2))#forword
    teF.extend(x_hash.cpu().data.numpy().tolist()) #record feature
    x_type = F.log_softmax(x_type,dim=1) 
    teY_prob.extend(x_type.cpu().data.numpy().tolist())
    pred = x_type.max(1,keepdim=True)[1]
    teY_pred.extend(pred.cpu().data.numpy().tolist())
    sys.stdout.write('\r {} / {} '.format(i, num_batches))
    sys.stdout.flush()
    
#performance of retrieval
# buliding index of trainset
tstart = time.time()
cpu_index = faiss.IndexFlatL2(36) #
gpu_index = faiss.index_cpu_to_all_gpus(cpu_index) #make all gpu usable
gpu_index.add(np.ascontiguousarray(trF, dtype=np.float32)) #add data(must be float32) to index
elapsed = time.time() - tstart    
print('Completed buliding index in %d seconds' % int(elapsed))
for topk in [5,10,15,20]:
    MHR = [] #mean Hit ratio 
    MAP = [] #mean average precision
    MRR = [] #mean reciprocal rank
    scores, neighbors = gpu_index.search(np.array(teF).astype('float32'), k=topk)
    for i, teVal in enumerate(teF):
        stype = teY[i]
        #perfromance
        pos_len = 0
        rank_len = 0
        mrr_flag = 0
        #for j in ranklist:
        for j in neighbors[i].tolist():
            dtype = trY[j]
            rank_len=rank_len+1
            if stype==dtype:  #hit
                MHR.append(1)
                pos_len = pos_len +1
                MAP.append(pos_len/rank_len) 
                if mrr_flag==0: 
                    MRR.append(pos_len/rank_len)
                    mrr_flag =1
            else: 
                MHR.append(0)
                MAP.append(0)   
    print("topk={}, mHR={:.6f}, mAP={:.6f}, mRR={:.6f}".format(topk, np.mean(MHR),np.mean(MAP),np.mean(MRR)))
#performance of classification
print ( 'Accuracy: %.6f'%accuracy_score(teY, teY_pred))
labels = list(set(teY))
cm = confusion_matrix(teY, teY_pred, labels=labels )
print (cm)
print ('Sensitivity of Normal: %.6f'%float(cm[0][0]/np.sum(cm[0])))
print ('Sensitivity of Edema: %.6f'%float(cm[1][1]/np.sum(cm[1])))
print ('Sensitivity of Pneumonia: %.6f'%float(cm[2][2]/np.sum(cm[2])))
print ('Sensitivity of Fracture: %.6f'%float(cm[3][3]/np.sum(cm[3])))

 1000 / 1000 : loss = 3.393445Eopch:     1 mean_loss = 3.654553
 1000 / 1000 : loss = 0.058874Eopch:     2 mean_loss = 2.197043
 1000 / 1000 : loss = 0.086843Eopch:     3 mean_loss = 1.818961
 1000 / 1000 : loss = 7.015194Eopch:     4 mean_loss = 1.498876
 1000 / 1000 : loss = 1.539314Eopch:     5 mean_loss = 1.252285
 1000 / 1000 : loss = 0.244521Eopch:     6 mean_loss = 1.055028
 1000 / 1000 : loss = 0.026969Eopch:     7 mean_loss = 0.800839
 1000 / 1000 : loss = 0.004753Eopch:     8 mean_loss = 0.645815
 1000 / 1000 : loss = 0.080672Eopch:     9 mean_loss = 0.516862
 1000 / 1000 : loss = 14.829358Eopch:    10 mean_loss = 0.488338
best_loss = 0.488338
 1999 / 2000 Completed buliding index in 1 seconds
topk=5, mHR=0.747860, mAP=0.721664, mRR=0.933217
topk=10, mHR=0.750270, mAP=0.716717, mRR=0.908965
topk=15, mHR=0.751197, mAP=0.714871, mRR=0.895957
topk=20, mHR=0.751415, mAP=0.713724, mRR=0.886979
Accuracy: 0.124200
[[ 346 1275 3835 9398]
 [  12  179  235  518]
 [  32  661 1659 1436]


In [13]:
for cls in labels:
    teY_cls = np.array(teY)[np.where( np.array(teY) == cls)[0]]
    teF_cls = np.array(teF)[np.where( np.array(teY) == cls)[0]]
    MHR = [] #mean Hit ratio 
    MAP = [] #mean average precision
    MRR = 0.0 #mean reciprocal rank
    scores, neighbors = gpu_index.search(np.array(teF_cls).astype('float32'), k=10)
    for i, teVal in enumerate(teF_cls):#perfromance
        stype = teY_cls[i]
        pos_len = 0
        rank_len = 0
        mrr_flag = 0
        for j in neighbors[i].tolist():
            dtype = trY[j]
            rank_len=rank_len+1
            if stype==dtype:  #hit
                MHR.append(1)
                pos_len = pos_len +1
                MAP.append(pos_len/rank_len) 
                if mrr_flag==0: 
                    MRR= pos_len/rank_len
                    mrr_flag = 1
            else: 
                MHR.append(0)
                MAP.append(0)   
    print("label={}, mHR={:.6f}, mAP={:.6f}, mRR={:.6f}".format(cls, np.mean(MHR),np.mean(MAP),np.mean(MRR)))

label=0, mHR=0.782180, mAP=0.752045, mRR=1.000000
label=1, mHR=0.343432, mAP=0.277156, mRR=1.000000
label=2, mHR=0.807049, mAP=0.765643, mRR=0.200000
label=3, mHR=0.013527, mAP=0.003807, mRR=0.250000


In [16]:
#define model
model = ATHNet(hash_size=36, type_size=4).cuda()#initialize model
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) #define optimizer
criterion  = HashLossFunc(margin=0.5).cuda() #define PairwiseLoss 
#train model
best_net, best_loss = None, float('inf')
batchSize = 10
for epoch in range(10):#iteration
    trI1_sf, trI2_sf, trY_sf = onlineGenImgPairs()
    losses = []
    num_batches = len(trY_sf) // batchSize 
    for i in range(num_batches):
        optimizer.zero_grad()#grad vanish
        min_idx = i * batchSize
        max_idx = np.min([len(trY_sf), (i+1)*batchSize])
        I1_batch = torch.from_numpy(trI1_sf[min_idx:max_idx]).type(torch.FloatTensor).cuda()
        I2_batch = torch.from_numpy(trI2_sf[min_idx:max_idx]).type(torch.FloatTensor).cuda()
        Y_batch = torch.from_numpy(trY_sf[min_idx:max_idx]).type(torch.FloatTensor).cuda()
        #forword
        X1_batch,_ = model(I1_batch.permute(0, 3, 1, 2))#permute the dims of matrix
        X2_batch,_ = model(I2_batch.permute(0, 3, 1, 2))
        #binary-like loss
        loss = criterion(X1_batch,X2_batch,Y_batch)
        #backward
        loss.backward()
        #update parameters
        optimizer.step()
        #show loss
        sys.stdout.write('\r {} / {} : loss = {}'.format(i+1, num_batches, float('%0.6f'%loss.item())))
        sys.stdout.flush()     
        losses.append(loss.item())
    print("Eopch: %5d mean_loss = %.6f" % (epoch + 1, np.mean(losses)))
    if np.mean(losses) < best_loss:
        best_loss = np.mean(losses)
        best_net = copy.deepcopy(model)
print("best_loss = %.6f" % (best_loss))

#release gpu memory
model = model.cpu()
tl_loss=tl_loss.cpu()
ce_loss=ce_loss.cpu()
torch.cuda.empty_cache()
#hash code of train data from model
#torch.cuda.synchronize()
batchSize = 10
num_batches = len(trI) // batchSize 
trF = []
for i in range(num_batches):
    min_idx = i * batchSize
    max_idx = np.min([len(trI), (i+1)*batchSize])
    I_batch = torch.from_numpy(np.array(trI[min_idx: max_idx])).type(torch.FloatTensor).cuda()
    X_batch,_ = best_net(I_batch.permute(0, 3, 1, 2))#forword
    I_batch = I_batch.cpu()
    X_batch = X_batch.cpu()
    torch.cuda.empty_cache()#release gpu memory
    trF.extend(X_batch.data.numpy().tolist())
    sys.stdout.write('\r {} / {} '.format(i, num_batches))
    sys.stdout.flush()
    
#hash code of test data from model
#torch.cuda.synchronize()
teY_pred = []
teY_prob = []
teF = [] 
num_batches = len(teY) // batchSize 
for i in range(num_batches):
    min_idx = i * batchSize
    max_idx = np.min([len(teY), (i+1)*batchSize])
    x_batch = torch.from_numpy(np.array(teI[min_idx:max_idx])).type(torch.FloatTensor).cuda()
    x_hash, x_type = best_net(x_batch.permute(0, 3, 1, 2))#forword
    teF.extend(x_hash.cpu().data.numpy().tolist()) #record feature
    x_type = F.log_softmax(x_type,dim=1) 
    teY_prob.extend(x_type.cpu().data.numpy().tolist())
    pred = x_type.max(1,keepdim=True)[1]
    teY_pred.extend(pred.cpu().data.numpy().tolist())
    sys.stdout.write('\r {} / {} '.format(i, num_batches))
    sys.stdout.flush()
    
#performance of retrieval
# buliding index of trainset
tstart = time.time()
cpu_index = faiss.IndexFlatL2(36) #
gpu_index = faiss.index_cpu_to_all_gpus(cpu_index) #make all gpu usable
gpu_index.add(np.ascontiguousarray(trF, dtype=np.float32)) #add data(must be float32) to index
elapsed = time.time() - tstart    
print('Completed buliding index in %d seconds' % int(elapsed))
for topk in [5,10,15,20]:
    MHR = [] #mean Hit ratio 
    MAP = [] #mean average precision
    MRR = [] #mean reciprocal rank
    scores, neighbors = gpu_index.search(np.array(teF).astype('float32'), k=topk)
    for i, teVal in enumerate(teF):
        stype = teY[i]
        #perfromance
        pos_len = 0
        rank_len = 0
        mrr_flag = 0
        #for j in ranklist:
        for j in neighbors[i].tolist():
            dtype = trY[j]
            rank_len=rank_len+1
            if stype==dtype:  #hit
                MHR.append(1)
                pos_len = pos_len +1
                MAP.append(pos_len/rank_len) 
                if mrr_flag==0: 
                    MRR.append(pos_len/rank_len)
                    mrr_flag =1
            else: 
                MHR.append(0)
                MAP.append(0)   
    print("topk={}, mHR={:.6f}, mAP={:.6f}, mRR={:.6f}".format(topk, np.mean(MHR),np.mean(MAP),np.mean(MRR)))
#performance of classification
print ( 'Accuracy: %.6f'%accuracy_score(teY, teY_pred))
labels = list(set(teY))
cm = confusion_matrix(teY, teY_pred, labels=labels )
print (cm)
print ('Sensitivity of Normal: %.6f'%float(cm[0][0]/np.sum(cm[0])))
print ('Sensitivity of Edema: %.6f'%float(cm[1][1]/np.sum(cm[1])))
print ('Sensitivity of Pneumonia: %.6f'%float(cm[2][2]/np.sum(cm[2])))
print ('Sensitivity of Fracture: %.6f'%float(cm[3][3]/np.sum(cm[3])))

 1000 / 1000 : loss = 0.210501Eopch:     1 mean_loss = 1.306045
 1000 / 1000 : loss = 0.477685Eopch:     2 mean_loss = 0.888655
 1000 / 1000 : loss = 1.85638Eopch:     3 mean_loss = 0.858163
 1000 / 1000 : loss = 0.171504Eopch:     4 mean_loss = 0.818934
 1000 / 1000 : loss = 1.033973Eopch:     5 mean_loss = 0.790585
 1000 / 1000 : loss = 1.021456Eopch:     6 mean_loss = 0.774501
 1000 / 1000 : loss = 0.31713Eopch:     7 mean_loss = 0.737395
 1000 / 1000 : loss = 0.129243Eopch:     8 mean_loss = 0.745025
 1000 / 1000 : loss = 0.750228Eopch:     9 mean_loss = 0.700849
 1000 / 1000 : loss = 0.570241Eopch:    10 mean_loss = 0.714397
best_loss = 0.700849
 1999 / 2000 Completed buliding index in 1 seconds
topk=5, mHR=0.748680, mAP=0.717284, mRR=0.926671
topk=10, mHR=0.747445, mAP=0.706423, mRR=0.902467
topk=15, mHR=0.747130, mAP=0.701632, mRR=0.889925
topk=20, mHR=0.746910, mAP=0.699054, mRR=0.883101
Accuracy: 0.185900
[[ 2872 11423    12   547]
 [  226   686     2    30]
 [ 1688  1372    2

In [17]:
for cls in labels:
    teY_cls = np.array(teY)[np.where( np.array(teY) == cls)[0]]
    teF_cls = np.array(teF)[np.where( np.array(teY) == cls)[0]]
    MHR = [] #mean Hit ratio 
    MAP = [] #mean average precision
    MRR = 0.0 #mean reciprocal rank
    scores, neighbors = gpu_index.search(np.array(teF_cls).astype('float32'), k=10)
    for i, teVal in enumerate(teF_cls):#perfromance
        stype = teY_cls[i]
        pos_len = 0
        rank_len = 0
        mrr_flag = 0
        for j in neighbors[i].tolist():
            dtype = trY[j]
            rank_len=rank_len+1
            if stype==dtype:  #hit
                MHR.append(1)
                pos_len = pos_len +1
                MAP.append(pos_len/rank_len) 
                if mrr_flag==0: 
                    MRR= pos_len/rank_len
                    mrr_flag = 1
            else: 
                MHR.append(0)
                MAP.append(0)   
    print("label={}, mHR={:.6f}, mAP={:.6f}, mRR={:.6f}".format(cls, np.mean(MHR),np.mean(MAP),np.mean(MRR)))

label=0, mHR=0.777898, mAP=0.738831, mRR=1.000000
label=1, mHR=0.123517, mAP=0.073226, mRR=0.500000
label=2, mHR=0.859556, mAP=0.812353, mRR=1.000000
label=3, mHR=0.051691, mAP=0.018219, mRR=0.200000


In [20]:
#define model
model = ATHNet(hash_size=36, type_size=4).cuda()#initialize model
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) #define optimizer
criterion  = CircleLoss().cuda() #define TripletLoss 
#train model
#train model
best_net, best_loss = None, float('inf')
batchSize = 10
trI = np.array(trI)
trY = np.array(trY)
for epoch in range(10):#iteration
    losses = []
    shuffled_idx = np.random.permutation(np.arange(len(trY)))
    train_x = trI[shuffled_idx]
    train_y = trY[shuffled_idx]
    num_batches = len(trY) // batchSize
    for i in range(num_batches):
        optimizer.zero_grad()#grad vanish
        min_idx = i * batchSize
        max_idx = np.min([len(trY), (i+1)*batchSize])
        x_batch = torch.from_numpy(train_x[min_idx:max_idx]).type(torch.FloatTensor).cuda()
        y_batch = torch.from_numpy(train_y[min_idx:max_idx]).type(torch.LongTensor).cuda()
        #forword
        out_batch,_ = model(x_batch.permute(0, 3, 1, 2))#permute the dims of matrix
        loss = criterion(out_batch,y_batch) #F.log_softmax+F.nll_loss
        #backward
        loss.backward()
        #update parameters
        optimizer.step()
        #show loss
        sys.stdout.write('\r {} / {} : loss = {}'.format(i+1, num_batches, float('%0.6f'%loss.item())))
        sys.stdout.flush()     
        losses.append(loss.item())
    print("Eopch: %5d mean_loss = %.6f" % (epoch + 1, np.mean(losses)))
    if np.mean(losses) < best_loss:
        best_loss = np.mean(losses)
        best_net = copy.deepcopy(model)
print("best_loss = %.6f" % (best_loss))
#release gpu memory
model = model.cpu()
criterion=criterion.cpu()
torch.cuda.empty_cache()

#hash code of train data from model
batchSize = 10
num_batches = len(trI) // batchSize
trF = []
for i in range(num_batches):
    min_idx = i * batchSize
    max_idx = np.min([len(trI), (i+1)*batchSize])
    I_batch = torch.from_numpy(np.array(trI[min_idx: max_idx])).type(torch.FloatTensor).cuda()
    X_batch,_ = best_net(I_batch.permute(0, 3, 1, 2))#forword
    I_batch = I_batch.cpu()
    X_batch = X_batch.cpu()
    torch.cuda.empty_cache()#release gpu memory
    trF.extend(X_batch.data.numpy().tolist())
    sys.stdout.write('\r {} / {} '.format(i, num_batches))
    sys.stdout.flush()
    
#hash code of test data from model
#torch.cuda.synchronize()
teF = []
num_batches = len(teI) // batchSize 
for i in range(num_batches):
    min_idx = i * batchSize
    max_idx = np.min([len(teI), (i+1)*batchSize])
    I_batch = torch.from_numpy(np.array(teI[min_idx: max_idx])).type(torch.FloatTensor).cuda()
    X_batch,_ = best_net(I_batch.permute(0, 3, 1, 2))#forword
    I_batch = I_batch.cpu()
    X_batch = X_batch.cpu()
    torch.cuda.empty_cache()#release gpu memory
    teF.extend(X_batch.data.numpy().tolist())
    sys.stdout.write('\r {} / {} '.format(i, num_batches))
    sys.stdout.flush()

# buliding index of trainset
tstart = time.time()
cpu_index = faiss.IndexFlatL2(36) #
gpu_index = faiss.index_cpu_to_all_gpus(cpu_index) #make all gpu usable
gpu_index.add(np.ascontiguousarray(trF, dtype=np.float32)) #add data(must be float32) to index
elapsed = time.time() - tstart    
print('Completed buliding index in %d seconds' % int(elapsed))
for topk in [5,10,15,20]:
    MHR = [] #mean Hit ratio 
    MAP = [] #mean average precision
    MRR = [] #mean reciprocal rank
    scores, neighbors = gpu_index.search(np.array(teF).astype('float32'), k=topk)
    for i, teVal in enumerate(teF):#perfromance
        stype = teY[i]
        pos_len = 0
        rank_len = 0
        mrr_flag = 0
        for j in neighbors[i].tolist():
            dtype = trY[j]
            rank_len=rank_len+1
            if stype==dtype:  #hit
                MHR.append(1)
                pos_len = pos_len +1
                MAP.append(pos_len/rank_len) 
                if mrr_flag==0: 
                    MRR.append(pos_len/rank_len)
                    mrr_flag =1
            else: 
                MHR.append(0)
                MAP.append(0)   
    print("topk={}, mHR={:.6f}, mAP={:.6f}, mRR={:.6f}".format(topk, np.mean(MHR),np.mean(MAP),np.mean(MRR)))
#performance
scores, neighbors = gpu_index.search(np.ascontiguousarray(teF, dtype=np.float32), k=1) #return top1
y_pred = []
for i in neighbors.flatten():
    y_pred.append(np.array(trY)[i]) #label of top1
print ( 'Accuracy: %.6f'%accuracy_score(teY, y_pred))
labels = list(set(teY))
cm = confusion_matrix(teY, y_pred, labels=labels )
print (cm)
print ('Sensitivity of Normal: %.6f'%float(cm[0][0]/np.sum(cm[0])))
print ('Sensitivity of Edema: %.6f'%float(cm[1][1]/np.sum(cm[1])))
print ('Sensitivity of Pneumonia: %.6f'%float(cm[2][2]/np.sum(cm[2])))
print ('Sensitivity of Fracture: %.6f'%float(cm[3][3]/np.sum(cm[3])))

 2000 / 2000 : loss = 0.8732189Eopch:     1 mean_loss = 6.135985
 2000 / 2000 : loss = 0.0211238Eopch:     2 mean_loss = 4.730092
 2000 / 2000 : loss = 10.220989Eopch:     3 mean_loss = 4.424806
 2000 / 2000 : loss = 3.8150846Eopch:     4 mean_loss = 3.960394
 2000 / 2000 : loss = 0.5865719Eopch:     5 mean_loss = 3.854856
 2000 / 2000 : loss = 1.0194685Eopch:     6 mean_loss = 3.628097
 2000 / 2000 : loss = 0.9292346Eopch:     7 mean_loss = 3.508072
 2000 / 2000 : loss = 3.5984125Eopch:     8 mean_loss = 3.285920
 2000 / 2000 : loss = 8.3393984Eopch:     9 mean_loss = 3.180617
 2000 / 2000 : loss = 7.2266784Eopch:    10 mean_loss = inf
best_loss = 3.180617
 1999 / 2000 Completed buliding index in 1 seconds
topk=5, mHR=0.758270, mAP=0.745401, mRR=0.962952
topk=10, mHR=0.758370, mAP=0.740716, mRR=0.943948
topk=15, mHR=0.758323, mAP=0.738289, mRR=0.932298
topk=20, mHR=0.758617, mAP=0.737108, mRR=0.924433
Accuracy: 0.760200
[[11679   407  2726    42]
 [  543   141   257     3]
 [  264   1

In [21]:
for cls in labels:
    teY_cls = np.array(teY)[np.where( np.array(teY) == cls)[0]]
    teF_cls = np.array(teF)[np.where( np.array(teY) == cls)[0]]
    MHR = [] #mean Hit ratio 
    MAP = [] #mean average precision
    MRR = 0.0 #mean reciprocal rank
    scores, neighbors = gpu_index.search(np.array(teF_cls).astype('float32'), k=10)
    for i, teVal in enumerate(teF_cls):#perfromance
        stype = teY_cls[i]
        pos_len = 0
        rank_len = 0
        mrr_flag = 0
        for j in neighbors[i].tolist():
            dtype = trY[j]
            rank_len=rank_len+1
            if stype==dtype:  #hit
                MHR.append(1)
                pos_len = pos_len +1
                MAP.append(pos_len/rank_len) 
                if mrr_flag==0: 
                    MRR= pos_len/rank_len
                    mrr_flag = 1
            else: 
                MHR.append(0)
                MAP.append(0)   
    print("label={}, mHR={:.6f}, mAP={:.6f}, mRR={:.6f}".format(cls, np.mean(MHR),np.mean(MAP),np.mean(MRR)))

label=0, mHR=0.786906, mAP=0.771929, mRR=1.000000
label=1, mHR=0.146398, mAP=0.103561, mRR=1.000000
label=2, mHR=0.879857, mAP=0.857105, mRR=1.000000
label=3, mHR=0.018357, mAP=0.008742, mRR=0.125000


In [24]:
#define model
model = ATHNet(hash_size=36, type_size=4).cuda()#initialize model
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) #define optimizer
criterion  = FocalLoss(gamma=2,alpha=[0.2,0.4,0.2,0.15,0.05]).cuda() #define ce mutli-classes, 
#train model
best_net, best_loss = None, float('inf')
batchSize = 10
trI = np.array(trI)
trY = np.array(trY)
for epoch in range(10):#iteration
    losses = []
    shuffled_idx = np.random.permutation(np.arange(len(trY)))
    train_x = trI[shuffled_idx]
    train_y = trY[shuffled_idx]
    num_batches = len(trY) // batchSize
    for i in range(num_batches):
        optimizer.zero_grad()#grad vanish
        min_idx = i * batchSize
        max_idx = np.min([len(trY), (i+1)*batchSize])
        x_batch = torch.from_numpy(train_x[min_idx:max_idx]).type(torch.FloatTensor).cuda()
        y_batch = torch.from_numpy(train_y[min_idx:max_idx]).type(torch.LongTensor).cuda()
        #forword
        _, out_batch = model(x_batch.permute(0, 3, 1, 2))#permute the dims of matrix
        loss = criterion(out_batch,y_batch) #F.log_softmax+F.nll_loss
        #backward
        loss.backward()
        #update parameters
        optimizer.step()
        #show loss
        sys.stdout.write('\r {} / {} : loss = {}'.format(i+1, num_batches, float('%0.6f'%loss.item())))
        sys.stdout.flush()     
        losses.append(loss.item())
    print("Eopch: %5d mean_loss = %.6f" % (epoch + 1, np.mean(losses)))
    if np.mean(losses) < best_loss:
        best_loss = np.mean(losses)
        best_net = copy.deepcopy(model)
print("best_loss = %.6f" % (best_loss))

#release gpu memory
model = model.cpu()
criterion = criterion.cpu()
torch.cuda.empty_cache()
#torch.cuda.synchronize()
batchSize = 10
num_batches = len(trI) // batchSize
trF = []
for i in range(num_batches):
    min_idx = i * batchSize
    max_idx = np.min([len(trI), (i+1)*batchSize])
    I_batch = torch.from_numpy(np.array(trI[min_idx: max_idx])).type(torch.FloatTensor).cuda()
    feature, _ = best_net(I_batch.permute(0, 3, 1, 2))#forword
    I_batch = I_batch.cpu()
    feature = feature.cpu()
    torch.cuda.empty_cache()#release gpu memory
    trF.extend(feature.data.numpy().tolist())
    sys.stdout.write('\r {} / {} '.format(i, num_batches))
    sys.stdout.flush()

teY_pred = []
teF = [] 
num_batches = len(teY) // batchSize 
for i in range(num_batches):
    min_idx = i * batchSize
    max_idx = np.min([len(teY), (i+1)*batchSize])
    x_batch = torch.from_numpy(np.array(teI[min_idx:max_idx])).type(torch.FloatTensor).cuda()
    feature, out_batch = best_net(x_batch.permute(0, 3, 1, 2))#forword
    teF.extend(feature.cpu().data.numpy().tolist()) #record feature
    out_batch = F.log_softmax(out_batch,dim=1) 
    pred = out_batch.max(1,keepdim=True)[1]
    teY_pred.extend(pred.cpu().data.numpy().tolist())
    sys.stdout.write('\r {} / {} '.format(i, num_batches))
    sys.stdout.flush()
    
# buliding index of trainset
tstart = time.time()
cpu_index = faiss.IndexFlatL2(36) #
gpu_index = faiss.index_cpu_to_all_gpus(cpu_index) #make all gpu usable
gpu_index.add(np.ascontiguousarray(trF, dtype=np.float32)) #add data(must be float32) to index
elapsed = time.time() - tstart    
print('Completed buliding index in %d seconds' % int(elapsed))
for topk in [5,10,15,20]:
    MHR = [] #mean Hit ratio 
    MAP = [] #mean average precision
    MRR = [] #mean reciprocal rank
    scores, neighbors = gpu_index.search(np.array(teF).astype('float32'), k=topk)
    for i, teVal in enumerate(teF):#perfromance
        stype = teY[i]
        pos_len = 0
        rank_len = 0
        mrr_flag = 0
        for j in neighbors[i].tolist():
            dtype = trY[j]
            rank_len=rank_len+1
            if stype==dtype:  #hit
                MHR.append(1)
                pos_len = pos_len +1
                MAP.append(pos_len/rank_len) 
                if mrr_flag==0: 
                    MRR.append(pos_len/rank_len)
                    mrr_flag =1
            else: 
                MHR.append(0)
                MAP.append(0)   
    print("topk={}, mHR={:.6f}, mAP={:.6f}, mRR={:.6f}".format(topk, np.mean(MHR),np.mean(MAP),np.mean(MRR)))
#performance
scores, neighbors = gpu_index.search(np.ascontiguousarray(teF, dtype=np.float32), k=1) #return top1
y_pred = []
for i in neighbors.flatten():
    y_pred.append(np.array(trY)[i]) #label of top1
print ( 'Accuracy: %.6f'%accuracy_score(teY, y_pred))
labels = list(set(teY))
cm = confusion_matrix(teY, y_pred, labels=labels )
print (cm)
print ('Sensitivity of Normal: %.6f'%float(cm[0][0]/np.sum(cm[0])))
print ('Sensitivity of Edema: %.6f'%float(cm[1][1]/np.sum(cm[1])))
print ('Sensitivity of Pneumonia: %.6f'%float(cm[2][2]/np.sum(cm[2])))
print ('Sensitivity of Fracture: %.6f'%float(cm[3][3]/np.sum(cm[3])))

 2000 / 2000 : loss = 0.090194Eopch:     1 mean_loss = 0.018596
 2000 / 2000 : loss = 0.000774Eopch:     2 mean_loss = 0.011180
 2000 / 2000 : loss = 0.010479Eopch:     3 mean_loss = 0.009744
 2000 / 2000 : loss = 0.001551Eopch:     4 mean_loss = 0.008319
 2000 / 2000 : loss = 0.022994Eopch:     5 mean_loss = 0.007469
 2000 / 2000 : loss = 0.001793Eopch:     6 mean_loss = 0.006831
 2000 / 2000 : loss = 0.016303Eopch:     7 mean_loss = 0.006214
 2000 / 2000 : loss = 0.010128Eopch:     8 mean_loss = 0.005845
 2000 / 2000 : loss = 0.001841Eopch:     9 mean_loss = 0.005276
 2000 / 2000 : loss = 0.011183Eopch:    10 mean_loss = 0.005058
best_loss = 0.005058
 1999 / 2000 Completed buliding index in 1 seconds
topk=5, mHR=0.744360, mAP=0.709509, mRR=0.920276
topk=10, mHR=0.742235, mAP=0.696114, mRR=0.894587
topk=15, mHR=0.741500, mAP=0.690030, mRR=0.880695
topk=20, mHR=0.740827, mAP=0.686240, mRR=0.871135
Accuracy: 0.747400
[[12602   833  1362    57]
 [  634   120   189     1]
 [ 1431    96  2

In [25]:
for cls in labels:
    teY_cls = np.array(teY)[np.where( np.array(teY) == cls)[0]]
    teF_cls = np.array(teF)[np.where( np.array(teY) == cls)[0]]
    MHR = [] #mean Hit ratio 
    MAP = [] #mean average precision
    MRR = 0.0 #mean reciprocal rank
    scores, neighbors = gpu_index.search(np.array(teF_cls).astype('float32'), k=10)
    for i, teVal in enumerate(teF_cls):#perfromance
        stype = teY_cls[i]
        pos_len = 0
        rank_len = 0
        mrr_flag = 0
        for j in neighbors[i].tolist():
            dtype = trY[j]
            rank_len=rank_len+1
            if stype==dtype:  #hit
                MHR.append(1)
                pos_len = pos_len +1
                MAP.append(pos_len/rank_len) 
                if mrr_flag==0: 
                    MRR= pos_len/rank_len
                    mrr_flag = 1
            else: 
                MHR.append(0)
                MAP.append(0)   
    print("label={}, mHR={:.6f}, mAP={:.6f}, mRR={:.6f}".format(cls, np.mean(MHR),np.mean(MAP),np.mean(MRR)))

label=0, mHR=0.861573, mAP=0.822008, mRR=1.000000
label=1, mHR=0.097669, mAP=0.054774, mRR=1.000000
label=2, mHR=0.513516, mAP=0.437300, mRR=1.000000
label=3, mHR=0.022947, mAP=0.009587, mRR=0.500000


In [28]:
#define model
model = ATHNet(hash_size=36, type_size=4).cuda()#initialize model
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) #define optimizer
criterion  = nn.CrossEntropyLoss().cuda() #define ce mutli-classes
#train model
best_net, best_loss = None, float('inf')
batchSize = 10
trI = np.array(trI)
trY = np.array(trY)
for epoch in range(10):#iteration
    losses = []
    shuffled_idx = np.random.permutation(np.arange(len(trY)))
    train_x = trI[shuffled_idx]
    train_y = trY[shuffled_idx]
    num_batches = len(trY) // batchSize
    for i in range(num_batches):
        optimizer.zero_grad()#grad vanish
        min_idx = i * batchSize
        max_idx = np.min([len(trY), (i+1)*batchSize])
        x_batch = torch.from_numpy(train_x[min_idx:max_idx]).type(torch.FloatTensor).cuda()
        y_batch = torch.from_numpy(train_y[min_idx:max_idx]).type(torch.LongTensor).cuda()
        #forword
        _,out_batch = model(x_batch.permute(0, 3, 1, 2))#permute the dims of matrix
        loss = criterion(out_batch,y_batch) #F.log_softmax+F.nll_loss
        #backward
        loss.backward()
        #update parameters
        optimizer.step()
        #show loss
        sys.stdout.write('\r {} / {} : loss = {}'.format(i+1, num_batches, float('%0.6f'%loss.item())))
        sys.stdout.flush()     
        losses.append(loss.item())
    print("Eopch: %5d mean_loss = %.6f" % (epoch + 1, np.mean(losses)))
    if np.mean(losses) < best_loss:
        best_loss = np.mean(losses)
        best_net = copy.deepcopy(model)
print("best_loss = %.6f" % (best_loss))

#release gpu memory
model = model.cpu()
criterion = criterion.cpu()
torch.cuda.empty_cache()
#torch.cuda.synchronize()
batchSize = 10
num_batches = len(trI) // batchSize
trF = []
for i in range(num_batches):
    min_idx = i * batchSize
    max_idx = np.min([len(trI), (i+1)*batchSize])
    I_batch = torch.from_numpy(np.array(trI[min_idx: max_idx])).type(torch.FloatTensor).cuda()
    feature, _ = best_net(I_batch.permute(0, 3, 1, 2))#forword
    I_batch = I_batch.cpu()
    feature = feature.cpu()
    torch.cuda.empty_cache()#release gpu memory
    trF.extend(feature.data.numpy().tolist())
    sys.stdout.write('\r {} / {} '.format(i, num_batches))
    sys.stdout.flush()

teY_pred = []
teF = [] 
num_batches = len(teY) // batchSize 
for i in range(num_batches):
    min_idx = i * batchSize
    max_idx = np.min([len(teY), (i+1)*batchSize])
    x_batch = torch.from_numpy(np.array(teI[min_idx:max_idx])).type(torch.FloatTensor).cuda()
    feature, out_batch = best_net(x_batch.permute(0, 3, 1, 2))#forword
    teF.extend(feature.cpu().data.numpy().tolist()) #record feature
    out_batch = F.log_softmax(out_batch,dim=1) 
    pred = out_batch.max(1,keepdim=True)[1]
    teY_pred.extend(pred.cpu().data.numpy().tolist())
    sys.stdout.write('\r {} / {} '.format(i, num_batches))
    sys.stdout.flush()

# buliding index of trainset
tstart = time.time()
cpu_index = faiss.IndexFlatL2(36) #
gpu_index = faiss.index_cpu_to_all_gpus(cpu_index) #make all gpu usable
gpu_index.add(np.ascontiguousarray(trF, dtype=np.float32)) #add data(must be float32) to index
elapsed = time.time() - tstart    
print('Completed buliding index in %d seconds' % int(elapsed))
for topk in [5,10,15,20]:
    MHR = [] #mean Hit ratio 
    MAP = [] #mean average precision
    MRR = [] #mean reciprocal rank
    scores, neighbors = gpu_index.search(np.array(teF).astype('float32'), k=topk)
    for i, teVal in enumerate(teF):#perfromance
        stype = teY[i]
        pos_len = 0
        rank_len = 0
        mrr_flag = 0
        for j in neighbors[i].tolist():
            dtype = trY[j]
            rank_len=rank_len+1
            if stype==dtype:  #hit
                MHR.append(1)
                pos_len = pos_len +1
                MAP.append(pos_len/rank_len) 
                if mrr_flag==0: 
                    MRR.append(pos_len/rank_len)
                    mrr_flag =1
            else: 
                MHR.append(0)
                MAP.append(0)   
    print("topk={}, mHR={:.6f}, mAP={:.6f}, mRR={:.6f}".format(topk, np.mean(MHR),np.mean(MAP),np.mean(MRR)))
#performance
scores, neighbors = gpu_index.search(np.ascontiguousarray(teF, dtype=np.float32), k=1) #return top1
y_pred = []
for i in neighbors.flatten():
    y_pred.append(np.array(trY)[i]) #label of top1
print ( 'Accuracy: %.6f'%accuracy_score(teY, y_pred))
labels = list(set(teY))
cm = confusion_matrix(teY, y_pred, labels=labels )
print (cm)
print ('Sensitivity of Normal: %.6f'%float(cm[0][0]/np.sum(cm[0])))
print ('Sensitivity of Edema: %.6f'%float(cm[1][1]/np.sum(cm[1])))
print ('Sensitivity of Pneumonia: %.6f'%float(cm[2][2]/np.sum(cm[2])))
print ('Sensitivity of Fracture: %.6f'%float(cm[3][3]/np.sum(cm[3])))

 2000 / 2000 : loss = 1.001225Eopch:     1 mean_loss = 0.168930
 2000 / 2000 : loss = 0.004424Eopch:     2 mean_loss = 0.106988
 2000 / 2000 : loss = 0.104603Eopch:     3 mean_loss = 0.094448
 2000 / 2000 : loss = 0.009696Eopch:     4 mean_loss = 0.084006
 2000 / 2000 : loss = 0.687054Eopch:     5 mean_loss = 0.078320
 2000 / 2000 : loss = 0.185454Eopch:     6 mean_loss = 0.072163
 2000 / 2000 : loss = 0.010744Eopch:     7 mean_loss = 0.067127
 2000 / 2000 : loss = 0.024265Eopch:     8 mean_loss = 0.064203
 2000 / 2000 : loss = 0.018953Eopch:     9 mean_loss = 0.060996
 2000 / 2000 : loss = 0.001119Eopch:    10 mean_loss = 0.057120
best_loss = 0.057120
 1999 / 2000 Completed buliding index in 1 seconds
topk=5, mHR=0.701590, mAP=0.663227, mRR=0.906416
topk=10, mHR=0.701515, mAP=0.649574, mRR=0.870855
topk=15, mHR=0.701643, mAP=0.643353, mRR=0.852944
topk=20, mHR=0.702002, mAP=0.639781, mRR=0.841875
Accuracy: 0.706950
[[11445  1874  1487    48]
 [  652   158   132     2]
 [ 1077   148  2

In [29]:
for cls in labels:
    teY_cls = np.array(teY)[np.where( np.array(teY) == cls)[0]]
    teF_cls = np.array(teF)[np.where( np.array(teY) == cls)[0]]
    MHR = [] #mean Hit ratio 
    MAP = [] #mean average precision
    MRR = 0.0 #mean reciprocal rank
    scores, neighbors = gpu_index.search(np.array(teF_cls).astype('float32'), k=10)
    for i, teVal in enumerate(teF_cls):#perfromance
        stype = teY_cls[i]
        pos_len = 0
        rank_len = 0
        mrr_flag = 0
        for j in neighbors[i].tolist():
            dtype = trY[j]
            rank_len=rank_len+1
            if stype==dtype:  #hit
                MHR.append(1)
                pos_len = pos_len +1
                MAP.append(pos_len/rank_len) 
                if mrr_flag==0: 
                    MRR= pos_len/rank_len
                    mrr_flag = 1
            else: 
                MHR.append(0)
                MAP.append(0)   
    print("label={}, mHR={:.6f}, mAP={:.6f}, mRR={:.6f}".format(cls, np.mean(MHR),np.mean(MAP),np.mean(MRR)))

label=0, mHR=0.782813, mAP=0.734045, mRR=1.000000
label=1, mHR=0.154237, mAP=0.102608, mRR=0.200000
label=2, mHR=0.592793, mAP=0.524021, mRR=1.000000
label=3, mHR=0.027295, mAP=0.014792, mRR=0.333333


In [27]:
model = model.cpu()
tl_loss=tl_loss.cpu()
ce_loss=ce_loss.cpu()
torch.cuda.empty_cache()